# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet
#tfdet.core.util.set_seed(777) #set seed
device = tfdet.core.util.select_device(0) #set device

# 1. Init Dataset

In [3]:
import os
path = "./balloon"
if not os.path.exists(path):
    !wget -qq --no-check-certificate http://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
    !unzip -qq balloon_dataset.zip
print(tfdet.dataset.util.tree_dir(path))

balloon/
    train/
        605521662_a470fef77f_b.jpg
        9330497995_4cf0438cb6_k.jpg
        5178670692_63a4365c9c_b.jpg
        ...
    val/
        4838031651_3e7b5ea5c7_b.jpg
        16335852991_f55de7958d_k.jpg
        24631331976_defa3bb61f_k.jpg
        ...


In [4]:
import numpy as np

image_shape = [256, 256]

label = tfdet.dataset.balloon.LABEL #["background", "balloon"]
n_class = len(label)

tr_path = os.path.join(path, "train/via_region_data.json")
te_path = os.path.join(path, "val/via_region_data.json")

def mask_func(mask_true):
    return np.max(mask_true, axis = 0)

for filename in os.listdir("./"):
    name, ext = os.path.splitext(filename)
    if name in ["train_cache", "sample_cache"]:# and (ext == ".index" or ".data" in ext):
        os.remove(filename)

tr_pipe = tfdet.dataset.balloon.load_pipe(tr_path, mask = True)
tr_pipe = tfdet.dataset.pipeline.load(tr_pipe, mask_func = mask_func)
tr_pipe = tfdet.dataset.pipeline.args2dict(tr_pipe)
if True: #If the number of training data is small, it is better to apply weak augmentation without spatial transform, or offline augmentation.
    tr_pipe = tfdet.dataset.pipeline.weak_augmentation(tr_pipe.cache("./train_cache"), crop_shape = None) #If crop_shape is shape or ratio, apply random_crop.
    tr_pipe = tfdet.dataset.pipeline.resize(tr_pipe, image_shape = image_shape, keep_ratio = True)
elif True:
    #tr_pipe = tfdet.dataset.pipeline.albumentations(tr_pipe)
    #tr_pipe = tfdet.dataset.pipeline.mmdet_augmentation(tr_pipe.cache("./train_cache"), image_shape = [int(min(image_shape) * 0.1), int(max(image_shape) * 2)], crop_shape = image_shape, resize_mode = "range", keep_ratio = True, shape_divisor = 32)
    tr_pipe = tfdet.dataset.pipeline.mmdet_augmentation(tr_pipe.cache("./train_cache"), image_shape = image_shape, crop_shape = None, keep_ratio = True, shape_divisor = 32)
else:
    tr_pipe = tfdet.dataset.pipeline.resize(tr_pipe, image_shape = image_shape, keep_ratio = True)
    tr_pipe = tfdet.dataset.pipeline.yolo_augmentation(tr_pipe.cache("./train_cache"), sample_x_true = tr_pipe.cache("./sample_cache"), p_mix_up = 0.15, p_copy_paste = 0.)
tr_pipe = tfdet.dataset.pipeline.pad(tr_pipe, image_shape = image_shape, mode = "both")
tr_pipe = tfdet.dataset.pipeline.filter_annotation(tr_pipe, min_scale = 2, min_instance_area = 1)
tr_pipe = tfdet.dataset.pipeline.collect(tr_pipe, keys = ["x_true", "mask_true"])
tr_pipe = tfdet.dataset.pipeline.label_encode(tr_pipe, label = tfdet.dataset.balloon.LABEL, one_hot = True, label_smoothing = 0)
tr_pipe = tfdet.dataset.pipeline.normalize(tr_pipe, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375])
tr_pipe = tfdet.dataset.pipeline.cast(tr_pipe)

te_pipe = tfdet.dataset.balloon.load_pipe(te_path, mask = True)
te_pipe = tfdet.dataset.pipeline.load(te_pipe, mask_func = mask_func)
te_pipe = tfdet.dataset.pipeline.args2dict(te_pipe)
te_pipe = tfdet.dataset.pipeline.resize(te_pipe, image_shape = image_shape, keep_ratio = True)
te_pipe = tfdet.dataset.pipeline.pad(te_pipe, image_shape = image_shape, mode = "both")
te_pipe = tfdet.dataset.pipeline.filter_annotation(te_pipe, min_scale = 2, min_instance_area = 1)
te_pipe = tfdet.dataset.pipeline.collect(te_pipe, keys = ["x_true", "mask_true"])
te_pipe = tfdet.dataset.pipeline.label_encode(te_pipe, label = tfdet.dataset.balloon.LABEL, one_hot = True, label_smoothing = 0)
te_pipe = tfdet.dataset.pipeline.normalize(te_pipe, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375])
te_pipe = tfdet.dataset.pipeline.cast(te_pipe)

# 2. Build Detector

In [5]:
with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
    feature = tfdet.model.backbone.resnet50(x, weights = "imagenet")
    
    out = tfdet.model.detector.upernet(feature, n_class = n_class)
    out = tf.keras.layers.UpSampling2D((4, 4))(out)
    model = tf.keras.Model(x, out)
    model.add_loss(lambda: tf.reduce_sum(tfdet.core.loss.regularize(model, weight_decay = 1e-4), name = "regularize_loss"))

# 3.Train

3-0. Init HyperParameter

In [6]:
import os, shutil

#epoch = 300
epoch = 50
train_batch_size = 4
batch_size = 16
save_path = "./learn/epoch@{epoch:03d}-metric@{mean_iou:.4f}-loss@{loss:.4f}-val_loss@{val_loss:.4f}.h5"

learning_rate = 1e-2
dst_learning_rate = 1e-4
momentum = 0.9
nesterov = True
warm_up_epoch = 5

if os.path.exists(os.path.dirname(save_path)):
    shutil.rmtree(os.path.dirname(save_path))
os.makedirs(os.path.dirname(save_path), exist_ok = True)

train_pipe = tfdet.dataset.pipeline.dict2args(tr_pipe, batch_size = train_batch_size, shuffle = True, prefetch = True)
test_pipe = tfdet.dataset.pipeline.dict2args(te_pipe, batch_size = batch_size, prefetch = True)

metric = tfdet.callback.MeanIoU(test_pipe, label = label, name = "mean_iou")
save = tf.keras.callbacks.ModelCheckpoint(save_path, monitor = "mean_iou", mode = "max", save_best_only = True, save_weights_only = True, save_freq = "epoch", verbose = 0)
scheduler = tfdet.callback.WarmUpCosineLearningRateScheduler(cycle = epoch, decay_rate = dst_learning_rate / learning_rate, total_step = None, warm_up_epoch = warm_up_epoch) #total step None > dynamic total step
logger = tf.keras.callbacks.CSVLogger(os.path.join(os.path.dirname(save_path), "logger.csv"), separator = ",")

3-1. Default

In [7]:
optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov)
model.compile(loss = tf.keras.losses.categorical_crossentropy, optimizer = optimizer)

with device:
    model.fit(train_pipe.repeat(8), validation_data = test_pipe, epochs = epoch, callbacks = [metric, save, scheduler, logger], verbose = 2)
    
for filename in os.listdir("./"):
    name, ext = os.path.splitext(filename)
    if name in ["train_cache", "sample_cache"]:# and (ext == ".index" or ".data" in ext):
        os.remove(filename)

Epoch 1/50

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.2358  |0.2333|0.3783|0.3783|
|balloon   |0.8184  |0.0595|0.1122|0.1122|
+----------+--------+------+------+------+
|summary   |0.5271  |0.1464|0.2453|0.2453|
+----------+--------+------+------+------+
32/32 - 186s - loss: 0.6655 - val_loss: 0.9194 - mean_accuracy: 0.5271 - mean_iou: 0.1464 - mean_dice: 0.2453 - mean_f1: 0.2453 - learning_rate: 6.6000e-05 - 186s/epoch - 6s/step
Epoch 2/50

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.8674  |0.8561|0.9225|0.9225|
|balloon   |0.7786  |0.2422|0.39  |0.39  |
+----------+--------+------+------+------+
|summary   |0.823   |0.5491|0.6562|0.6562|
+----------+--------+------+------+------+
32/32 - 144s - loss: 0.2398 - val_loss: 0.4194 - mean_accuracy: 0.8230 - mean_iou: 0.5491 - mean_dice: 0.6562

3-2. Stochastic Weight Averaging(SWA)

In [7]:
import traceback

optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov)
try:
    !pip install tensorflow_addons
    import tensorflow_addons as tfa
    #optimizer = tfa.optimizers.SGDW(learning_rate = learning_rate, weight_decay = 1e-4, momentum = momentum, nesterov = nesterov)
    optimizer = tfa.optimizers.SWA(optimizer, start_averaging = epoch // 3, average_period = epoch // 10)
except:
    print(traceback.format_exc())
model.compile(loss = tf.keras.losses.categorical_crossentropy, optimizer = optimizer)

with device:
    model.fit(train_pipe.repeat(8), validation_data = test_pipe, epochs = epoch, callbacks = [metric, save, scheduler, logger], verbose = 2)
    
for filename in os.listdir("./"):
    name, ext = os.path.splitext(filename)
    if name in ["train_cache", "sample_cache"]:# and (ext == ".index" or ".data" in ext):
        os.remove(filename)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_addons-0.18.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
Epoch 1/50

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.2951  |0.2941|0.4546|0.4546|
|balloon   |0.9437  |0.0739|0.1376|0.1376|
+----------+--------+------+------+------+
|summary   |0.6194  |0.184 |0.2961|0.2961|
+----------+--------+------+------+------+
32/32 - 191s - loss: 0.7924 - val_loss: 0.8305 - mean_accuracy: 0.6194 - mean_iou: 0.1840 - mean_dice: 0.2961 - mean_f1: 0.2961 - learning_rate: 6.6000e-05 - 191s/epoch - 6s/step
Epoch 2/50

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.9647  |0.9486|0.9736|0.9736|
|balloon   |0.7153  |0.4502|0.6209|0.6209|
+----------+--------+------+----

3-2. Exponential Moving Average(EMA)

In [7]:
optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov)
model.compile(loss = tf.keras.losses.categorical_crossentropy, optimizer = optimizer)

ema = tfdet.callback.EMA(step = 64 // batch_size, auto_apply = True, warm_up_epoch = warm_up_epoch)

with device:
    model.fit(train_pipe.repeat(8), validation_data = test_pipe, epochs = epoch, callbacks = [ema, metric, save, scheduler, logger], verbose = 2)
    
for filename in os.listdir("./"):
    name, ext = os.path.splitext(filename)
    if name in ["train_cache", "sample_cache"]:# and (ext == ".index" or ".data" in ext):
        os.remove(filename)

Epoch 1/50

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.0382  |0.0381|0.0734|0.0734|
|balloon   |0.9402  |0.0551|0.1044|0.1044|
+----------+--------+------+------+------+
|summary   |0.4892  |0.0466|0.0889|0.0889|
+----------+--------+------+------+------+
32/32 - 204s - loss: 0.6349 - val_loss: 1.3676 - mean_accuracy: 0.4892 - mean_iou: 0.0466 - mean_dice: 0.0889 - mean_f1: 0.0889 - learning_rate: 6.6000e-05 - 204s/epoch - 6s/step
Epoch 2/50

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.7779  |0.7649|0.8668|0.8668|
|balloon   |0.7149  |0.1518|0.2636|0.2636|
+----------+--------+------+------+------+
|summary   |0.7464  |0.4583|0.5652|0.5652|
+----------+--------+------+------+------+
32/32 - 149s - loss: 0.2214 - val_loss: 0.5421 - mean_accuracy: 0.7464 - mean_iou: 0.4583 - mean_dice: 0.5652

# 4. Evaluate

In [8]:
model_path = tfdet.dataset.util.list_dir(os.path.dirname(save_path), "h5")
model_path = sorted(model_path, key = lambda x: x.split("metric@")[1].split("-")[0], reverse = True)[0]
model.load_weights(model_path)

metric = tfdet.core.metric.MeanIoU(label = label)
for x_true, y_true in test_pipe:
    y_pred = model.predict(x_true, verbose = 0)
    metric.add(y_true, y_pred)
print(metric.summary_text)

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.9966  |0.9916|0.9958|0.9958|
|balloon   |0.9156  |0.8668|0.9287|0.9287|
+----------+--------+------+------+------+
|summary   |0.9561  |0.9292|0.9622|0.9622|
+----------+--------+------+------+------+


# 5. Load

In [9]:
with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
    feature = tfdet.model.backbone.resnet50(x, weights = None)

    out = tfdet.model.detector.upernet(feature, n_class = n_class)
    out = tf.keras.layers.UpSampling2D((4, 4))(out)
    model = tf.keras.Model(x, out)
    model.load_weights(model_path)
    
metric = tfdet.core.metric.MeanIoU(label = label)
for x_true, y_true in test_pipe:
    y_pred = model.predict(x_true, verbose = 0)
    metric.add(y_true, y_pred)
print(metric.summary_text)

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.9966  |0.9916|0.9958|0.9958|
|balloon   |0.9156  |0.8668|0.9287|0.9287|
+----------+--------+------+------+------+
|summary   |0.9561  |0.9292|0.9622|0.9622|
+----------+--------+------+------+------+
